Notes:
* The data is for two years: 2023 and 2024
* The issuer_bank_name has only value of "My Favorite Bank". Possible to drop
* Null values seem to be reasonable.
    merchant_id = null when payment is automatic like salary  
    merchant_mcc = null whenever merchant_id = null  
    merchant_city = null Almost always when merchant_id = null  
    original amount = null if no conversion to other currency  
    wallet_type = null if regular card is used  
    pos_entry_mode = unclear what null signifies. When it is null the transaction is either  
    "P2P_OUT", "BILL_PAYMENT", "SALARY", "P2P_IN"  
* There is cases (~739) where we have merchant_city but no merchant_id

In [2]:
import polars as pl

data = pl.read_parquet("data/DECENTRATHON_3.0.parquet")

In [3]:
data.head()

transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,datetime[ns],i64,str,str,i64,i64,str,str,str,f64,f64,str,str,str,str,i64
"""ffb21bbd-fef1-4983-9774-3b01c8…",2023-08-12 23:14:19.043176,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",1237.66,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",0
"""acd754f9-bd4c-416d-babc-926050…",2023-08-12 17:07:08.641030,10000,"""09/26""","""My Favorite Bank""",50359,5499,"""Grocery & Food Markets""","""Shymkent""","""POS""",2328.03,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",1
"""b023697b-eb35-40e6-af75-3586c6…",2023-08-12 08:45:32.946846,10000,"""09/26""","""My Favorite Bank""",50214,4814,"""Utilities & Bill Payments""","""Almaty""","""BILL_PAYMENT""",53871.78,null,"""KZT""","""KAZ""",null,null,2
"""d59d8537-1e48-41b5-91e1-dc7cf9…",2023-08-12 10:20:20.242510,10000,"""09/26""","""My Favorite Bank""",50279,8011,"""Pharmacies & Health""","""Shymkent""","""POS""",22501.56,null,"""KZT""","""KAZ""","""Contactless""","""Apple Pay""",3
"""85d2eef6-3b09-47a6-987f-f45ab4…",2023-08-12 00:42:00.556252,10000,"""09/26""","""My Favorite Bank""",50332,5499,"""Grocery & Food Markets""","""Almaty""","""POS""",1224.04,null,"""KZT""","""KAZ""","""QR_Code""","""Bank's QR""",4


In [4]:
data.describe()

statistic,transaction_id,transaction_timestamp,card_id,expiry_date,issuer_bank_name,merchant_id,merchant_mcc,mcc_category,merchant_city,transaction_type,transaction_amount_kzt,original_amount,transaction_currency,acquirer_country_iso,pos_entry_mode,wallet_type,__index_level_0__
str,str,str,f64,str,str,f64,f64,str,str,str,f64,f64,str,str,str,str,f64
"""count""","""11536850""","""11536850""",1.153685e7,"""11536850""","""11536850""",9.066814e6,9.066814e6,"""11536850""","""9067553""","""11536850""",1.153685e7,5417.0,"""11536850""","""11536850""","""8883438""","""6863356""",1.153685e7
"""null_count""","""0""","""0""",0.0,"""0""","""0""",2.470036e6,2.470036e6,"""0""","""2469297""","""0""",0.0,1.1531433e7,"""0""","""0""","""2653412""","""4673494""",0.0
"""mean""",null,"""2024-04-23 10:50:42.587606""",11003.031321,null,null,50249.453106,5644.688338,null,null,null,20166.686882,74316.000809,null,null,null,null,5768424.5
"""std""",null,null,580.572998,null,null,143.985954,1030.229349,null,null,null,81305.720721,575082.106378,null,null,null,null,3.3304e6
"""min""","""0000013f-41fa-4e8e-b029-df002c…","""2023-01-01 00:30:12.551745""",10000.0,"""01/25""","""My Favorite Bank""",50000.0,3000.0,"""Clothing & Apparel""","""Aktobe""","""ATM_WITHDRAWAL""",100.0,0.21,"""AED""","""ARE""","""Chip""","""Apple Pay""",0.0
"""25%""",null,"""2023-12-21 14:19:07.908075""",10494.0,null,null,50126.0,5200.0,null,null,null,2140.45,28.8,null,null,null,null,2.884212e6
"""50%""",null,"""2024-05-19 05:29:53.788712""",10998.0,null,null,50248.0,5499.0,null,null,null,6070.66,164.78,null,null,null,null,5.768425e6
"""75%""",null,"""2024-09-17 17:55:15.468680""",11504.0,null,null,50375.0,5812.0,null,null,null,19259.91,2086.07,null,null,null,null,8.652637e6
"""max""","""fffffe8e-4a4f-4ba7-990c-19c995…","""2024-12-31 23:59:35.437993""",11999.0,"""12/29""","""My Favorite Bank""",50499.0,9311.0,"""Utilities & Bill Payments""","""Shymkent""","""SALARY""",3.4073e7,2.7190e7,"""UZS""","""UZB""","""Swipe""","""Samsung Pay""",1.1536849e7


In [7]:
total_transactions = data.get_column("transaction_amount_kzt").sum()

transactions_by_category = (data.group_by("mcc_category")
                                .agg(pl.col("transaction_amount_kzt").sum())
                                .with_columns(
                                    (pl.col("transaction_amount_kzt")/total_transactions).alias("share_in_ttl_trnsc")))


transactions_by_category.plot.bar(x="mcc_category", y = "share_in_ttl_trnsc")

alt.Chart(...)

In [8]:
transactions_by_type = (data.group_by("transaction_type")
                                .agg(pl.col("transaction_amount_kzt").sum())
                                .with_columns(
                                    (pl.col("transaction_amount_kzt")/total_transactions).alias("share_in_ttl_trnsc"))
                                .sort(by="share_in_ttl_trnsc", descending=True))

transactions_by_type.plot.bar(x="transaction_type", y = "share_in_ttl_trnsc")

alt.Chart(...)

In [49]:
reference_date = pl.lit("2025-01-01").str.to_date()


clustering_data = data.group_by("card_id").agg(
    (reference_date - pl.col("transaction_timestamp").max()).dt.total_hours().alias("last_trans"),
    (pl.col("transaction_timestamp").max() - pl.col("transaction_timestamp").min()).dt.total_days().alias("use_dur"),
    (pl.col("transaction_amount_kzt").sum()).alias("total_trans"),
    (pl.col("transaction_amount_kzt").count()).alias("number_trans"),
    (pl.col("transaction_amount_kzt").mean()).alias("avg_trans")
).with_columns(
    daily_turnover = pl.col("number_trans")/pl.col("use_dur")
)

In [51]:
clustering_data

card_id,last_trans,use_dur,total_trans,number_trans,avg_trans,daily_turnover
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0


In [ ]:
dictionary = pl.DataFrame(
    schema={
        "column_name": pl.String,
        "description": pl.String
    }
)


def add_column_group(clustering_data, column, name, null_des = None):
    # creating a dataframe by merchant category total tranactions
    column_transactions = data.pivot(
        values="transaction_amount_kzt",
        index="card_id",
        on=column,
        aggregate_function="sum").fill_null(0).rename(lambda c: f"{name}_{c}" if c !="card_id" else c)

    # adding description to dictionary
    if null_des:
        types = [f"{"name"}_{c}" for c in data.get_column(column).unique()]
        descriptions = [f"total transactions by {type_}" if type_ else f"total transactions by {null_des}" for type_ in types]
        dictionary.vstack(pl.DataFrame({"column_name": types,
                                        "description": descriptions}),
                        in_place=True)
    else:
        types = [f"{"name"}_{c}" for c in data.get_column(column).unique()]
        descriptions = [f"total transactions by {type_}" for type_ in types]
        dictionary.vstack(pl.DataFrame({"column_name": types,
                                        "description": descriptions}),
                        in_place=True)
        
    # joining the column transactions
    return clustering_data.join(column_transactions, on="card_id", how="left")
        

In [ ]:
clustering_data = add_column_group(clustering_data, "wallet_type", "wallet", "regular card")

In [71]:
look_up = pl.read_parquet("data/clustering_data.parquet")
look_up

card_id,last_trans,use_dur,total_trans,number_trans,avg_trans,daily_turnover,pos_Contactless,pos_QR_Code,pos_null,pos_ECOM,pos_Swipe,pos_Chip,type_POS,type_BILL_PAYMENT,type_ECOM,type_ATM_WITHDRAWAL,type_P2P_OUT,type_P2P_IN,type_SALARY,category_Grocery & Food Markets,category_Utilities & Bill Payments,category_Pharmacies & Health,category_General Retail & Department,category_Unknown,category_Services (Other),category_Electronics & Software,category_Dining & Restaurants,category_Clothing & Apparel,category_Travel & Transportation,"category_Hobby, Books, Sporting Goods",category_Fuel & Service Stations,category_Home Furnishings & Supplies,wallet_Apple Pay,wallet_Bank's QR,wallet_null,wallet_Samsung Pay,wallet_Google Pay,in_KZT,in_TRY,in_CNY,in_AED,in_AMD,in_BYN,in_KGS,in_UZS,in_USD,in_GEL,in_EUR
i64,i64,i64,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10260,0,25,1.2131e7,862,14073.649652,34.48,5.2399e6,3.5246e6,0.0,599045.17,662766.99,2.1052e6,8.8304e6,0.0,599045.17,2.702e6,0.0,0.0,0.0,1.1483e6,825538.47,85266.94,1.0138e6,2.702e6,1.7330e6,591216.55,639835.96,1.3085e6,1332471.6,11900.5,596732.96,142930.21,931262.85,3.5246e6,4.0370e6,1988663.4,1.6500e6,1.2131e7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11960,7336,160,4.6901e7,2639,17772.234494,16.49375,9.9617e6,8.0929e6,1.3308e7,6.3541e6,710406.26,8.4734e6,1.9598e7,6.7623e6,6.3541e6,7.641e6,2.3329e6,2328647.6,1.884476e6,1.4465e6,8.6215e6,135502.04,1.9188e6,1.4187e7,4.9350e6,7.9532e6,195874.86,4.2271e6,1.0474e6,401123.28,1.2101e6,621814.4,3.5701e6,8.0929e6,2.6124e7,4.6991e6,4414657.8,4.6901e7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11013,1,679,2.9265e8,18003,16255.828334,26.513991,2.1544e7,2.1923e7,1.0650e8,1.2581e8,486503.71,1.6390e7,2.9829e7,1.2888e7,1.2581e8,3.0514e7,4.6697e7,4.2552e7,4.365944e6,1.1219e7,1.4697e7,996037.74,7.1283e7,1.2413e8,2.2946e7,1.0023e7,1.6498e6,2.8941e6,7.8050e6,630595.96,2.0563e7,3.8188e6,4.9164e7,2.1923e7,1.3939e8,3.8012e7,4.4165e7,2.9245e8,10154.7,0.0,10550.6,30959.39,31141.81,14018.04,55020.3,25167.23,0.0,28633.5
11016,0,688,5.6008e8,18026,31070.575257,26.200581,2.6147e7,5.2130e7,1.2384e8,3.3830e8,1.8481e6,1.7808e7,6.9325e7,2.7906e7,3.3830e8,2.8608e7,4.5299e7,4.6119e7,4.519116e6,4.5624e6,5.3521e7,6.1834e6,4.2119e6,1.2455e8,2.0843e8,1.0454e7,4.0821e6,3.4714e6,1.2318e8,1.2077e7,1.4088e6,3.9570e6,1.3854e8,5.2130e7,1.5731e8,1.0487e8,1.0722e8,5.5812e8,18772.18,100799.46,981913.77,0.0,337510.78,119342.96,203627.95,92356.47,52138.73,54181.01
10644,7334,283,5.9993e6,162,37032.774815,0.572438,779030.86,501126.89,3.3850e6,317011.66,7612.2,1.0095e6,780316.56,842768.86,317011.66,1.517e6,61996.67,236305.77,2.24391e6,92157.99,842768.86,4688.25,29964.23,4.0592e6,186820.47,355937.94,5375.21,127011.56,260588.78,0.0,34783.79,0.0,28559.68,501126.89,5.0801e6,156578.83,232904.01,5.9993e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
11376,735,49,2.5154e6,245,10267.081592,5.0,229000.0,91710.96,429715.74,1.4950e6,0.0,270000.0,91710.96,218343.74,1.4950e6,499000.0,0.0,0.0,211372.0,92211.99,231078.9,144411.07,872510.57,710372.0,64942.81,57256.75,16589.61,16447.81,27037.59,15320.48,267255.41,0.0,670012.64,91710.96,928715.74,410035.92,414959.73,2.5154e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10090,0,504,1.8800e8,6292,29879.686637,12.484127,8.4616e7,5.7977e6,1.3631e7,0.0,480884.43,8.3478e7,1.2798e7,1.2043e7,0.0,1.61574e8,0.0,0.0,1.588063e6,1.5822e6,1.2251e7,21187.58,211230.57,1.63162063e8,8.8865e6,595253.91,66010.29,77513.8,532372.46,17707.69,71374.63,528470.21,1.3176e6,5.7977e6,1.7729e8,1.5739e6,2.0236e6,1.8800e8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10054,2,637,2.1538e8,5631,38249.132566,8.839874,1.1886e7,1.5700e7,1.6350e8,1.7028e7,315874.25,6.9548e6,2.5095e7,1.5813e8,1.7028e7,9.761e6,0.0,0.0,5.362414e6,5.1662e6,1.5834e8,691843.27,1.1384e7,1.5123414e7